In [14]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('./data/08_wine.csv')
df.head(3)

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [9]:
y = df['class'].to_numpy()
x = df.drop('class', axis=1).to_numpy()

In [15]:
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2, random_state=42)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((5197, 3), (1300, 3), (5197,), (1300,))

In [17]:
dt = DecisionTreeClassifier()
dt

DecisionTreeClassifier()

In [18]:
dt.fit(train_x,train_y )

DecisionTreeClassifier()

In [22]:
print(dt.score(train_x,train_y ))
print(dt.score(test_x,test_y ))

0.996921300750433
0.8576923076923076


In [21]:
# cross_validate(생성모델명 ,훈련 독립변수, 훈련 종속변수)
# fit_time : 폴드별 훈련시간
# score_time : 폴드별 검증시간
# test_score : 폴드별 성능
cv = cross_validate(dt,train_x,train_y)
cv

{'fit_time': array([0.00500202, 0.00410914, 0.        , 0.        , 0.01595712]),
 'score_time': array([0., 0., 0., 0., 0.]),
 'test_score': array([0.86442308, 0.85096154, 0.87391723, 0.85178056, 0.83445621])}

In [23]:
# 교차검증 성능(폴드의 평균값)
np.mean(cv["test_score"])

0.8551077219219664

### 폴드(fold) 클래스 사용하기

In [24]:
# 폴드 컨트롤을 위한 라이브러리
# 훈련데이터 섞거나 폴드의 갯수를 지정
from sklearn.model_selection import StratifiedKFold
# n_splits :폴드 수
# shuffle : 반복학습 후 데이터를 섞을지 여부
# random_state : 난수씨드

In [25]:
# 폴드 클래스 사용
# cv : 분할기라고 칭하며 사용할 클래스를 지정함, 없는경우 기본 5폴드로 사용됨
ccv = cross_validate(dt,train_x, train_y , cv = StratifiedKFold())
ccv

{'fit_time': array([0.0045042 , 0.00350833, 0.00491142, 0.0039866 , 0.00398564]),
 'score_time': array([0.00050187, 0.00099874, 0.        , 0.00099778, 0.        ]),
 'test_score': array([0.87307692, 0.85096154, 0.87487969, 0.85178056, 0.83830606])}

In [26]:
# 훈련 정확도
np.mean(ccv["test_score"])

0.857800955060339

In [28]:
ccv.get

<function dict.get(key, default=None, /)>

In [29]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
ccv = cross_validate(dt,train_x, train_y , cv = splitter)
ccv

{'fit_time': array([0.00476861, 0.        , 0.        , 0.01751661, 0.        ,
        0.        , 0.01491737, 0.00498319, 0.00498319, 0.00498319]),
 'score_time': array([0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.00099945, 0.        , 0.        , 0.00099993]),
 'test_score': array([0.84038462, 0.88846154, 0.86153846, 0.85769231, 0.84615385,
        0.86923077, 0.85576923, 0.84585742, 0.85741811, 0.87861272])}

In [30]:
# 훈련 정확도
np.mean(ccv["test_score"])

0.86011190158589

### 하이퍼파라미터 튜닝(AutoML)

#### [AutoML]
#### - 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
#### - 사용패키지 : sklearn.model_selection
#### - 사용클래스 : GridSearchCV(max_dept, ...)

In [31]:
from sklearn.model_selection import GridSearchCV

In [39]:
## 하이퍼파라미터 속성 및 값 범위 설정
# 딕셔너리(키 : 밸류)로 생성
# 찾을 하이퍼파라미터 key값이 여러개인 경우 콤마로 구분하여 추가
param = {'max_depth': range(5,20,1)}

In [42]:
# 첫번째 : 훈련모델 알고리듬
# 두번째 : 찾을 하이퍼파라미터 지정
# 세번째 : 사용할 CPU 코어수로 -1이면 전체를 사용, 기본값은 1
# gnsfusqksqhrghlttn : 기본폴드 5 개 * max_dept 15개 = 75회 반복훈련
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 param_grid=param,
                 n_jobs = -1)
gs

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [44]:
ggs = gs.fit(train_x, train_y)

In [45]:
# 가장 좋은 하이퍼파라미터
ggs.best_params_

{'max_depth': 8}

In [47]:
# 가장 좋은 검증 성능 확인
ggs.best_score_

0.859729029392167

In [49]:
# 가장 좋은 하이퍼파라미터
ggs.best_estimator_

DecisionTreeClassifier(max_depth=8, random_state=42)

In [50]:
ggs.cv_results_

{'mean_fit_time': array([0.00152926, 0.        , 0.        , 0.00910077, 0.0128902 ,
        0.00300112, 0.00696411, 0.01094446, 0.00796051, 0.01033435,
        0.00876384, 0.00637884, 0.00637865, 0.00620136, 0.00639367]),
 'std_fit_time': array([0.00064097, 0.        , 0.        , 0.00744413, 0.00603657,
        0.00600224, 0.00853508, 0.00894352, 0.00974959, 0.00543961,
        0.00508532, 0.00048797, 0.00048832, 0.00040447, 0.00047626]),
 'mean_score_time': array([0.        , 0.        , 0.        , 0.0031817 , 0.        ,
        0.0001812 , 0.00019932, 0.00059786, 0.00339975, 0.00099726,
        0.000598  , 0.00079737, 0.00081029, 0.00059805, 0.00059791]),
 'std_score_time': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.67029965e-03,
        0.00000000e+00, 3.62396240e-04, 3.98635864e-04, 4.88149394e-04,
        5.82000769e-03, 1.57717089e-06, 4.88266288e-04, 3.98683847e-04,
        4.05409930e-04, 4.88362189e-04, 4.88188328e-04]),
 'param_max_depth': masked_array(data=

In [51]:
# 가장 성능이 좋은 하이퍼라미터 클래스를 받아서 사용
dt = ggs.best_estimator_
dt

DecisionTreeClassifier(max_depth=8, random_state=42)

In [53]:
# 훈련데이터 정확도 확인
dt.score(train_x, train_y)

0.9003271117952665

In [54]:
# 가장 좋은 검증 성능 확인( 기본 5폴드에 대한 성능 평균)
ggs.best_score_

0.859729029392167

#### AutoML이 수행한 모든 결과 확인
#### - mean_fit_time : 훈련 수행시간(15번)
#### - mean_score_time 검증데이터 훈련수행시간(15번)
#### - split0_test_score : 폴드별 성능

In [55]:
ggs.cv_results_

{'mean_fit_time': array([0.00152926, 0.        , 0.        , 0.00910077, 0.0128902 ,
        0.00300112, 0.00696411, 0.01094446, 0.00796051, 0.01033435,
        0.00876384, 0.00637884, 0.00637865, 0.00620136, 0.00639367]),
 'std_fit_time': array([0.00064097, 0.        , 0.        , 0.00744413, 0.00603657,
        0.00600224, 0.00853508, 0.00894352, 0.00974959, 0.00543961,
        0.00508532, 0.00048797, 0.00048832, 0.00040447, 0.00047626]),
 'mean_score_time': array([0.        , 0.        , 0.        , 0.0031817 , 0.        ,
        0.0001812 , 0.00019932, 0.00059786, 0.00339975, 0.00099726,
        0.000598  , 0.00079737, 0.00081029, 0.00059805, 0.00059791]),
 'std_score_time': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.67029965e-03,
        0.00000000e+00, 3.62396240e-04, 3.98635864e-04, 4.88149394e-04,
        5.82000769e-03, 1.57717089e-06, 4.88266288e-04, 3.98683847e-04,
        4.05409930e-04, 4.88362189e-04, 4.88188328e-04]),
 'param_max_depth': masked_array(data=

In [56]:
ggs.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

#### params의 값 확인

In [57]:
ggs.cv_results_['params']

[{'max_depth': 5},
 {'max_depth': 6},
 {'max_depth': 7},
 {'max_depth': 8},
 {'max_depth': 9},
 {'max_depth': 10},
 {'max_depth': 11},
 {'max_depth': 12},
 {'max_depth': 13},
 {'max_depth': 14},
 {'max_depth': 15},
 {'max_depth': 16},
 {'max_depth': 17},
 {'max_depth': 18},
 {'max_depth': 19}]

#### 첫번재 폴드의 검증결과 확인
#### - max_depth의 범위갯수만큼 나옴(15개)

In [58]:
ggs.cv_results_['split0_test_score']

array([0.84711538, 0.84807692, 0.85769231, 0.85288462, 0.85769231,
       0.84519231, 0.85865385, 0.86730769, 0.86538462, 0.86826923,
       0.86730769, 0.86442308, 0.86346154, 0.86634615, 0.87211538])

In [61]:
# 가장 좋았던 성능점수로 max_depth 가 8일때 가장 좋음
print(ggs.best_score_)   
# max_depth  평균
print(np.mean(ggs.cv_results_['mean_test_score']))

0.859729029392167
0.85722551269712


#### 교차검증값 조회

In [60]:
ggs.cv_results_['mean_test_score']

array([0.85780355, 0.8558801 , 0.85530133, 0.85972903, 0.85703228,
       0.85376471, 0.85838121, 0.85915007, 0.8572244 , 0.85799419,
       0.85626157, 0.8576105 , 0.85549308, 0.8578017 , 0.85895499])

In [62]:
# 훈련정확도, 테스트 정확도
ggs.best_score_, dt.score(test_x, test_y)

(0.859729029392167, 0.8584615384615385)